In [1]:
import sys
import os
import io
import django
import openai

# ---------------- UTF-8 Fix برای ویندوز ----------------
sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding="utf-8")

# ---------------- Django Setup ----------------
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
sys.path.append(BASE_DIR)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "travelproj.settings")

django.setup()

# ---------------- OpenAI Config ----------------
openai.api_key = "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"
openai.api_base = "https://openrouter.ai/api/v1"
MODEL = "qwen/qwen3-235b-a22b:free"

# ---------------- Imports ----------------
from tours.utils.query_searcher import search_tour_chunks
from tours.utils.query_filter import get_chunks_for_query


# ---------------- Build Prompt ----------------
def build_and_answer(user_query, filtered_chunks, retrieved, history):
    prompt = "📌 لیست تورهای مرتبط موجود است. از اطلاعات زیر استفاده کن:\n\n"

    # ---- نتایج فیلتر ----
    prompt += "✅ نتایج فیلتر:\n"
    for i, c in enumerate(filtered_chunks, 1):
        if hasattr(c, "text"):   # مدل Chunk
            prompt += f"{i}. {c.text}\n"
        elif isinstance(c, dict) and "chunk" in c:  # خروجی dict
            prompt += f"{i}. {c['chunk'].text}\n"
        else:
            prompt += f"{i}. {str(c)}\n"

    # ---- نتایج similarity search ----
    prompt += "\n🔎 نتایج مشابهت:\n"
    for i, r in enumerate(retrieved, 1):
        chunk = r["chunk"] if isinstance(r, dict) else r
        prompt += f"{i}. {chunk.text}\n"

    # ---- تاریخچه ----
    prompt += "\n🕑 تاریخچه گفتگو:\n"
    for turn in history:
        prompt += f"- {turn}\n"

    # ---- سوال جدید ----
    prompt += f"\n❓ سوال جدید کاربر: {user_query}\n"
    prompt += "پاسخ نهایی را ساده و دقیق بده."

    return prompt


# ---------------- Chat Loop ----------------
def manual_chat():
    history = []

    print("🤖 Chatbot آماده‌ست! بنویس، برای خروج 'exit' بزن.\n")

    while True:
        query = input("❓ پرسش: ")
        if query.strip().lower() in ["exit", "quit"]:
            print("👋 پایان گفتگو.")
            break

        # --- فیلتر و چانک‌ها
        filtered_tours, filtered_chunks = get_chunks_for_query(query)

        # --- similarity search
        retrieved = search_tour_chunks(query, top_k=5)

        # --- ساخت پرامپت
        answer = build_and_answer(query, filtered_chunks, retrieved, history)

        # --- نمایش
        print("\n💬 پاسخ:\n", answer)
        print("-" * 50)

        # --- ذخیره تاریخچه
        history.append({"role": "user", "content": query})
        history.append({"role": "assistant", "content": answer})


if __name__ == "__main__":
    manual_chat()


ModuleNotFoundError: No module named 'openai'